In [1]:
from copy import deepcopy
from functools import partial, reduce
from inspect import signature
import os
from operator import or_, and_, contains
import random as rand
import subprocess

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State, MATCH, ALL
from dash.exceptions import PreventUpdate
from dateutil.parser import parse
import django
from django.db.models import Q
import flask
from flask_caching import Cache
#from jupyter_plotly_dash import JupyterDash
import pandas as pd
import PIL
from PIL import Image
import plotly.express as px
import plotly.graph_objects as go
import pylibmc
from toolz import keyfilter, valfilter, merge, isiterable, get_in

# to make jupyter-plotly-dash work, had to install jupyter_server_proxy explicitly
# pip install jupyter_server_proxy
# jupyter serverextension enable jupyter_server_proxy
# but honestly it sort of isn't very good anyway and is not presently being used

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "mastspec.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from plotter.models import *
from plotter.graph import *
# from plotter.views import *
# from plotter.forms import *
from utils import (djget, modeldict, rows, columns, qlist, keygrab, in_me,
                   pickitems, pickctx, make_printer, dump_it, get_parameters)

In [2]:
app = dash.Dash()

In [3]:
def randfig():
    fig = go.Figure()
    fig.add_trace(
            go.Scattergl(
                y=[rand.randint(0,100) for i in range(100)],
                mode="lines",
            )
        )

In [4]:
def some_tab(index):
    return dcc.Tab(
        html.P(
            children = 'tab number ' + str(index),
            id=str(index)+'_text',     
        )
    )

In [5]:
app.layout = html.Div(children = [
    dcc.Tabs(
        children = [],
        id = 'tabs'
    ),
    html.Button('add tab', id='add-tab-button', n_clicks=0)
])

In [6]:
def add_app_tab(n_clicks, existing_tabs):
    output_tabs = deepcopy(existing_tabs)
    output_tabs.append(some_tab(n_clicks))
    return output_tabs

In [7]:
app.callback(
    Output('tabs', 'children'),
    [
        Input('add-tab-button', 'n_clicks')
    ],
    [
        State('tabs', 'children')
    ]
)(add_app_tab)

<function __main__.add_app_tab(n_clicks, existing_tabs)>

In [8]:
app.run_server()

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Aug/2020 04:47:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2020 04:47:49] "GET /_dash-component-suites/dash_renderer/react@16.v1_5_0m1592852348.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2020 04:47:49] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_5_0m1592852348.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2020 04:47:49] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_5_0m1592852348.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2020 04:47:49] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_10_1m1592852348.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2020 04:47:49] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_5_0m1592852348.13.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2020 04:47:49] "GET /_dash-component-suites/dash_core_components/dash_core_components.v1_10_1m1592852348.min.js HTTP/1.1" 200 -
127.0.0.1 - - [14/Aug/2020 0